In [1]:
import os
import google.generativeai as genai
from langchain.document_loaders import TextLoader                                           
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer

load_dotenv()

# Configure Gemini API
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# 1. Load and Process the Resume
resumeLoader = TextLoader("HariSamynaath_resume_20250210.txt")
resumeDocs = resumeLoader.load()
# split to chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
ResumeTexts = text_splitter.split_documents(resumeDocs)

# repeat same with elaborate write up
writeupLoader = TextLoader("HariSamynaath_profile_writeup.txt")
writeupDocs = writeupLoader.load()
# split to chunks
writeupTexts = text_splitter.split_documents(writeupDocs)

# collect all documents
texts = ResumeTexts
texts.extend(writeupTexts)

class SentenceTransformerEmbeddings:
    def __init__(self, model_name="all-mpnet-base-v2"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_tensor=False).tolist()

    def embed_query(self, text):
        return self.model.encode([text], convert_to_tensor=False).tolist()[0]
    
embeddings = SentenceTransformerEmbeddings("all-mpnet-base-v2")
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

db = Chroma.from_documents(texts, embeddings, persist_directory="./.chroma_db") #Local persistance.

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
retriever = db.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

def ask_question(question,wordlimit=100):
    presetText = f"""explain in less than {wordlimit} words as a continuous paragraph.
consider 'you' as addressing the resume owner.
Just provide only the answer.
answer in the voice of the resume owner."""
    query = '\n'.join([question, presetText])
    response = qa_chain.invoke(query)
    return response['result']

In [7]:
for doc in retriever.get_relevant_documents("certificate"):
    print(myWrap(doc.model_dump()['page_content']))

• Subject Matter Expert in sourcing, part development, and part & tool costing.
Managed 8 unique vendors and 8 unique part classes, comprising over 3000 SKUs
from 12 different models, with a total value of ₹1850 Cr.
• Led a team that quickly and effectively resolved a water leakage issue during
new source development, preventing field escalations and containing the problem
within factory limits.
• Led a team that contributed significantly to stabilizing a derailed
production line after a fire accident at a component supplier, through new-tool
tryouts, production planning, sequencing, and quality control. The stabilization
efforts spanned two weeks, successfully preventing a significant disruption to
the final assembly line.
Manufacturing Specialist | Hyundai Motor India Ltd. | Chennai, India | 2015 –
2018
• Subject Matter Expert in sourcing, part development, and part & tool costing.
Managed 8 unique vendors and 8 unique part classes, comprising over 3000 SKUs
from 12 different models,

In [4]:
def myWrap(text, width=80):
    lines = text.splitlines()
    wrapped_lines = []

    for line in lines:
        words = line.split()
        newLine = ""
        for word in words:
            if len(newLine) + len(word) + 1 <= width:
                newLine = ' '.join([newLine, word])
            else:
                wrapped_lines.append(newLine.strip())
                newLine = word
        
        wrapped_lines.append(newLine.strip())
        
    return '\n'.join(wrapped_lines)

for text in texts:
    myStr = text.model_dump()['page_content']
    print(myWrap(myStr))
    print(f"\n{len(myStr)} ", "="*10,end="\n\n")

len(texts)

questions = [
    "What are your key skills?",
    "What are your professional experiences?",
    "What projects have you worked on?",
    "What is your educational background?"
]

for question in questions:
    answer = ask_question(question)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

In [8]:
print(myWrap(ask_question("List your certificates")))

I don't have certificates listed in my resume. However, my skills encompass a
wide range of data science techniques, including machine learning, deep
learning, and NLP. I'm proficient with tools like Power BI, Azure Machine
Learning, Python, and SQL. My background also includes experience in project
management, process improvement, and leadership, demonstrated through roles at
Hyundai Motor India and Renault Nissan.
